<a href="https://colab.research.google.com/github/commd/08-2016-infinite-loops/blob/master/Selenium_Working_File_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
!pip install selenium
!apt-get update -qq
!apt install chromium-chromedriver -qq

     |████████████████████████████████| 911kB 4.8MB/s 
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 77.3 MB of archives.
After this operation, 264 MB of additional disk space will be used.
Selecting previously unselected package chromium-codecs-ffmpeg-extra.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../chromium-codecs-ffmpeg-extra_81.0.4044.138-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-codecs-ffmpeg-extra (81.0.4044.138-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-browser.
Preparing to unpack .../chromium-browser_81.0.

In [0]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


# Try out a few things

## Load the web page

https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p2

In [0]:
url_page = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/'

In [0]:
driver.get(url_page+'p2')

## Get one Comment, name, date and message

In [0]:
userid_element = driver.find_elements_by_xpath('//li[@id="Comment_1738047"]//a[@class="Username"]')[0]
userid = userid_element.text
userid

'wishnhigh1'

In [0]:
user_date = driver.find_elements_by_xpath("//li[@id='Comment_1738047']//a[@name='Item_51']//time")[0]
date = user_date.get_attribute('title')
date

'April 27, 2002 2:54PM'

In [0]:
user_message = driver.find_elements_by_xpath("//li[@id='Comment_1738047']//div[@class='Message userContent']")[0]
comment = user_message.text
comment

"The problem is that they are HUGE generalizations. Also, you are talking Japanese as almost exclusivly Honda and Toyota. What about Subaru, Mazda, Isuzu, Mitsubishi, Daihatsu, and Suzuki? So far the only makes that seem to keep with that Japanese Culture are Toyota and Honda.\n\nAnd many American and European makes are now capable of having higher quality than more than 70% of Japanese cars. Doesn't help your generalization that much, does it?\n\nSo where do the generalizations stop? They should stop with brands, and not nations."

## Let's generliaze for one comment by writing functions

In [0]:
comment_id = 'Comment_1738047'
comment_id

'Comment_1738047'

In [0]:
def get_userid(id):
    user_path = '//li[@id="' + id + '"]//a[@class="Username"]'
    return driver.find_elements_by_xpath(user_path)[0].text

get_userid(comment_id)

'techhawk'

In [0]:
def get_date(id):
  date_path = '//li[@id="' + id + '"]//a//time'
  return driver.find_elements_by_xpath(date_path)[0].get_attribute('title')

get_date(comment_id)

'September 16, 2002 2:38PM'

In [0]:
def get_message(id):
    message_path = '//li[@id="' + id + '"]//div[@class="Message userContent"]'
    return driver.find_elements_by_xpath(message_path)[0].text

get_message(comment_id)


"These are three pretty different cars, so it really comes down to what you value. The Passat is a real bargain with the nicest and largest interior as well as elegant exterior styling. I looked hard at a GLX and W8 when I was shopping. The handling is pretty wallowing, and the power is just adequate (I'm a leadfoot ).\n\nThe 325i is fun and rewarding to drive, really nice handling and ergonomics. Its pretty small and the price goes up fast as you add options, and its also underpowered for my taste.\n\nI've never driven the C240. The C320 I drove was nice, but not terribly sporty and I was disappointed in the quality of some of the interior pieces (they need to see where VW gets their switches).\n\nAll three a very good cars, I could be happy with any of them based on my needs."

## Let's get a list of all comment on a page

In [0]:
def get_comment_info(comment_id):
    #Extract dates from for each user on a page
    user_date = get_date(comment_id)

    #Extract user ids from each user on a page
    user_id = get_userid(comment_id)

    #Extract Message for each user on a page
    user_message = get_message(comment_id)                                      
   
    return [user_date, user_id, user_message]


print (comment_id, get_comment_info(comment_id))

Comment_1727067 ['September 16, 2002 2:38PM', 'techhawk', "These are three pretty different cars, so it really comes down to what you value. The Passat is a real bargain with the nicest and largest interior as well as elegant exterior styling. I looked hard at a GLX and W8 when I was shopping. The handling is pretty wallowing, and the power is just adequate (I'm a leadfoot ).\n\nThe 325i is fun and rewarding to drive, really nice handling and ergonomics. Its pretty small and the price goes up fast as you add options, and its also underpowered for my taste.\n\nI've never driven the C240. The C320 I drove was nice, but not terribly sporty and I was disappointed in the quality of some of the interior pieces (they need to see where VW gets their switches).\n\nAll three a very good cars, I could be happy with any of them based on my needs."]


In [0]:
# Create an empty data frame
import pandas as pd


In [0]:
# get a list of all comment_ids in the page
ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
comment_ids = []
for i in ids:
    comment_ids.append(i.get_attribute('id'))
comment_ids

In [0]:
comments = pd.DataFrame(columns = ['Date','UserID','Comment']) 

for comment_id  in comment_ids:
    #Adding date, userid and comment for each user in a dataframe  
    comments.loc[len(comments)] = get_comment_info(comment_id)

comments.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     50 non-null     object
 1   UserID   50 non-null     object
 2   Comment  50 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [0]:
comments.head()

,Date,UserID,Comment
0,"April 27, 2002 2:54PM",wishnhigh1,The problem is that they are HUGE generalizati...
1,"April 27, 2002 7:38PM",kd6aw1,Have found out that with some of the more powe...
2,"April 28, 2002 3:11AM",fwatson,How does your theory explain English cars? A c...
3,"April 29, 2002 7:59AM",dave330i,"""Being that it is an automatic I can enjoy my ..."
4,"April 29, 2002 10:12AM",blueguydotcom,"He did mention ""in rush hour traffic."" Like th..."


In [0]:
comments.tail()

,Date,UserID,Comment
45,"September 15, 2002 12:53AM",jagboyxtype,Some Acuras are experiencing very serious prob...
46,"September 16, 2002 7:06AM",jpnwdc,I am having the hardest time choosing a replac...
47,"September 16, 2002 12:29PM",kdshapiro,Add the following cons to the Mercedes and VW:...
48,"September 16, 2002 2:18PM",jpnwdc,kdshapiro - thanks for your post.\n\nI'm findi...
49,"September 16, 2002 2:38PM",techhawk,"These are three pretty different cars, so it r..."


## Let's generalize for a page

In [0]:
def get_comments_for_page(page_num):
    # load the page
    driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p'+str(page_num))

    #get the list of ids
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")

    #get comments for all ids in the page
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    for comment_id  in comment_ids:
      #Adding date, userid and comment for each user in a dataframe  
      comments_pd.loc[len(comments_pd)] = get_comment_info(comment_id)
    return 0




In [0]:
comments_pd = pd.DataFrame(columns = ['Date','UserId','Comments']) 
get_comments_for_page(2)
comments_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      50 non-null     object
 1   UserId    50 non-null     object
 2   Comments  50 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [0]:
comments_pd.head()

,Date,UserId,Comments
0,"April 27, 2002 2:54PM",wishnhigh1,The problem is that they are HUGE generalizati...
1,"April 27, 2002 7:38PM",kd6aw1,Have found out that with some of the more powe...
2,"April 28, 2002 3:11AM",fwatson,How does your theory explain English cars? A c...
3,"April 29, 2002 7:59AM",dave330i,"""Being that it is an automatic I can enjoy my ..."
4,"April 29, 2002 10:12AM",blueguydotcom,"He did mention ""in rush hour traffic."" Like th..."


In [0]:
comments_pd.tail()

,Date,UserId,Comments
45,"September 15, 2002 12:53AM",jagboyxtype,Some Acuras are experiencing very serious prob...
46,"September 16, 2002 7:06AM",jpnwdc,I am having the hardest time choosing a replac...
47,"September 16, 2002 12:29PM",kdshapiro,Add the following cons to the Mercedes and VW:...
48,"September 16, 2002 2:18PM",jpnwdc,kdshapiro - thanks for your post.\n\nI'm findi...
49,"September 16, 2002 2:38PM",techhawk,"These are three pretty different cars, so it r..."


# Get Complete from all pages

In [0]:
start_page = 2
end_page = 10
comments_pd = pd.DataFrame(columns = ['Date','UserId','Comments']) 
for i in range(start_page,end_page+1):
    get_comments_for_page(i)

comments_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      450 non-null    object
 1   UserId    450 non-null    object
 2   Comments  450 non-null    object
dtypes: object(3)
memory usage: 14.1+ KB


In [0]:
comments_pd.head()

,Date,UserId,Comments
0,"April 27, 2002 2:54PM",wishnhigh1,The problem is that they are HUGE generalizati...
1,"April 27, 2002 7:38PM",kd6aw1,Have found out that with some of the more powe...
2,"April 28, 2002 3:11AM",fwatson,How does your theory explain English cars? A c...
3,"April 29, 2002 7:59AM",dave330i,"""Being that it is an automatic I can enjoy my ..."
4,"April 29, 2002 10:12AM",blueguydotcom,"He did mention ""in rush hour traffic."" Like th..."


In [0]:
comments_pd.tail()

,Date,UserId,Comments
445,"January 10, 2004 1:33PM",saugatak,"riez, i agree with you 100%. you can't even ju..."
446,"January 11, 2004 6:47AM",kdshapiro,"""others don't have much but put all their cash..."
447,"January 15, 2004 8:58AM",mohatu,"What a rhetorical question... Of course, it's ..."
448,"January 15, 2004 9:33AM",kdshapiro,"""What a rhetorical question... Of course, it's..."
449,"January 15, 2004 9:46AM",shipo,"I'm hoping to be old, rich, healthy, driving a..."


In [0]:
comments_pd['UserId'].value_counts()